## Pairwise Evaluation

일부 평가에서는 두 개 이상의 LLM 생성물을 서로 비교하고자 합니다.

[Chatbot Arena](https://lmsys.org/blog/2023-05-03-arena/) 나 LLM 리더보드에서 어렵지 않게 접할 수 있는 비교 평가 방식입니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

이제 이러한 예제 실행에서 데이터셋을 생성할 수 있습니다.

입력만 저장하면 됩니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


def evaluate_pairwise(runs: list, example) -> dict:
    """
    A simple evaluator for pairwise answers to score based on  engagement
    """

    # 점수 저장
    scores = {}
    for i, run in enumerate(runs):
        scores[run.id] = i

    # 각 예제에 대한 실행 쌍
    answer_a = runs[0].outputs["answer"]
    answer_b = runs[1].outputs["answer"]
    question = example.inputs["question"]

    # 함수 호출이 있는 LLM, 최고 성능 모델 사용
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    # 구조화된 프롬프트
    grade_prompt = PromptTemplate.from_template(
        """
        You are an LLM judge. Compare the following two answers to a question and determine which one is better.
        Better answer is the one that is more detailed and informative.
        If the answer is not related to the question, it is not a good answer.
        
        # Question:
        {question}
        
        #Answer A: 
        {answer_a}
        
        #Answer B: 
        {answer_b}
        
        Output should be either `A` or `B`. Pick the answer that is better.
        
        #Preference:
        """
    )
    answer_grader = grade_prompt | llm | StrOutputParser()

    # 점수 획득
    score = answer_grader.invoke(
        {
            "question": question,
            "answer_a": answer_a,
            "answer_b": answer_b,
        }
    )
    # score = score["Preference"]

    # 점수에 따른 실행 할당 매핑
    if score == "A":  # Assistant A 선호
        scores[runs[0].id] = 1
        scores[runs[1].id] = 0
    elif score == "B":  # Assistant B 선호
        scores[runs[0].id] = 0
        scores[runs[1].id] = 1
    else:
        scores[runs[0].id] = 0
        scores[runs[1].id] = 0

    return {"key": "ranked_preference", "scores": scores}

비교 평가를 수행합니다.

In [ ]:
from langsmith.evaluation import evaluate_comparative

# 실험 이름 또는 ID 배열 교체
evaluate_comparative(
    ["MODEL_COMPARE_EVAL-e18f80ab", "MODEL_COMPARE_EVAL-7b929533"],
    # 평가자 배열
    evaluators=[evaluate_pairwise],
)

![](./assets/output-11.png)